In [1]:
import json
import pandas as pd

In [2]:
with open('/kaggle/input/benzinga-news/benzinga.json', "r") as file:
    newsfeeds = json.load(file)
len(newsfeeds)

12942

In [3]:
newsfeeds[0]

{'created': '2018-05-01',
 'title': "21 Stocks Moving In Tuesday's Pre-Market Session",
 'body': "Gainers Heat Biologics, Inc. (NASDAQ:HTBX) rose 44.9 percent to $3.10 in pre-market trading after climbing 87.72 percent on Monday. Inogen, Inc. (NASDAQ:INGN) rose 25.2 percent to $175.99 in pre-market trading following its first-quarter earnings report that saw FY18 sales guidance raised. Earnings came in at 48 cents per share, up from 27 cents per share last year. Sales came in at $79.1 million, beating estimates by $15 million. Karyopharm Therapeutics Inc. (NASDAQ:KPTI) rose 23.9 percent to $16.20 in pre-market trading following positive trial data from STORM study for selinexor. Nutrisystem, Inc. (NASDAQ:NTRI) shares rose 10.5 percent to $32.05 in pre-market trading after reporting better-than-expected Q1 earnings. Electronics for Imaging, Inc. (NASDAQ:EFII) rose 9.3 percent to $30.27 in pre-market trading following Q1 sales beat. Harmonic Inc. (NASDAQ:HLIT) rose 8.2 percent to $3.95 i

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")
if spacy.prefer_gpu():
    print("Working on GPU")
else:
    print("Working on CPU")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Working on GPU


In [5]:
news_data = []
for i in range(len(newsfeeds)):
    news_data.append(newsfeeds[i]['body'])

train_data = []
# Extract five required labels
for text in news_data:
    doc = nlp(text)
    for sent in doc.sents:
        entities = []
        for ent in sent.ents:
            if ent.label_ in ["PERSON", "GPE", "ORG", "PERCENT", "EVENT"]:
                # Calculate start and end character positions relative to the sentence, not the whole document
                start_char = ent.start_char - sent.start_char
                end_char = ent.end_char - sent.start_char
                entities.append((start_char, end_char, ent.label_))
        if entities:
            train_data.append((sent.text, {"entities": entities}))


In [6]:
import spacy
import random
from spacy.training.example import Example
from spacy.util import minibatch, compounding

nlp = spacy.blank("en")  # start with a blank Language class
nlp.add_pipe('ner')

ner = nlp.get_pipe("ner")

# Adding labels
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Initialize the weights and vocabulary
nlp.initialize()

random.seed(0)
spacy.util.fix_random_seed(0)

# Disable other components for training
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*unaffected_pipes):

    optimizer = nlp.initialize()

    for itn in range(35):
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            examples = [Example.from_dict(nlp.make_doc(text), annot) for text, annot in zip(texts, annotations)]
            nlp.update(examples, sgd=optimizer, drop=0.5, losses=losses) 
        # print every 10 loop
        if (itn + 1) % 10 == 0:
            print(f'Iteration {itn + 1}: Losses', losses)


[2023-07-29 08:59:04,566] [INFO] Created vocabulary
[2023-07-29 08:59:04,567] [INFO] Finished initializing nlp object
[2023-07-29 08:59:16,319] [INFO] Created vocabulary
[2023-07-29 08:59:16,320] [INFO] Finished initializing nlp object
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Market Reaction Regardless of the reason for the t..." with entities "[(68, 72, 'ORG'), (92, 96, 'ORG'), (97, 100, 'ORG'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Movers Indices S&P 500 ETF (NYSE:SPY) rose 2.79% t..." with entities "[(0, 22, 'ORG'), (28, 32, 'ORG'), (33, 36, 'ORG'),...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(te

Iteration 10: Losses {'ner': 165160.3046384073}
Iteration 20: Losses {'ner': 157999.82423298375}
Iteration 30: Losses {'ner': 155137.87941017025}


In [7]:
# Save the model
output_dir = "./model"
nlp.to_disk(output_dir)

In [8]:

# nlp = spacy.load("en_core_web_sm")
# ner = nlp.get_pipe("ner")

# for _, annotations in train_data:
#     for ent in annotations.get("entities"):
#         ner.add_label(ent[2])

# # Fixed random seed for reproducibility
# random.seed(0)
# spacy.util.fix_random_seed(0)

# # Disable other components for training
# unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
# with nlp.disable_pipes(*unaffected_pipes):

#     # Reduced number of iterations
#     for itn in range(50):
#         random.shuffle(train_data)
#         losses = {}
#         # Larger initial batch size
#         batches = minibatch(train_data, size=compounding(16., 64., 1.5))
#         for batch in batches:
#             texts, annotations = zip(*batch)
#             examples = [Example.from_dict(nlp.make_doc(text), annot) for text, annot in zip(texts, annotations)]
#             nlp.update(examples, drop=0.5, losses=losses) 
#         # print every loop
#         print(f'Iteration {itn + 1}: Losses', losses)

#         # Early stopping: stop if no improvement in last 10 iterations
#         if itn > 10 and losses['ner'] >= previous_losses['ner']:
#             break
#         previous_losses = losses

# # Export the trained model
# nlp.to_disk("./my_model")


In [9]:
# import spacy
# from spacy.pipeline import EntityRecognizer

# # nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load("en_core_web_md")


In [10]:
# # use first 20 articles to assure sufficient train data, data of one article is too small
# news_data = []
# for i in range(len(newsfeeds)):
#     news_data.append( newsfeeds[i]['body'])
    
# train_data = []
# # extract three required labels
# for text in news_data:
#     for sent in nlp(text).sents:
#         entities = []
#         for ent in sent.ents:
#             if ent.label_ in ["PERSON", "GPE", "ORG"]:
#                 entities.append((ent.start_char, ent.end_char, ent.label_))

#         if entities:
#             train_data.append((sent.text, {"entities": entities}))

In [11]:
# import spacy
# import random
# from spacy.training.example import Example
# from spacy.util import minibatch, compounding

# nlp = spacy.load("en_core_web_sm")
# ner = nlp.get_pipe("ner")

# for _, annotations in train_data:
#     for ent in annotations.get("entities"):
#         ner.add_label(ent[2])

# # Fixed random seed for reproducibility
# random.seed(0)
# spacy.util.fix_random_seed(0)

# # Disable other components for training
# unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
# with nlp.disable_pipes(*unaffected_pipes):
    
#     for itn in range(150):
#         random.shuffle(train_data)
#         losses = {}
#         batches = minibatch(train_data, size=compounding(4., 32., 1.001))
#         for batch in batches:
#             texts, annotations = zip(*batch)
#             examples = [Example.from_dict(nlp.make_doc(text), annot) for text, annot in zip(texts, annotations)]
#             nlp.update(examples, drop=0.5, losses=losses) 
#         # print every 10 loop
#         if (itn + 1) % 10 == 0:
#             print(f'Iteration {itn + 1}: Losses', losses)

In [12]:
# nlp.to_disk("./spacy-ner-mdoel")

In [13]:
# nlp = spacy.load("./spacy-ner-mdoel")

# doc = nlp("This is a sample text.")
# for ent in doc.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)